This notebook provides a simple sudoku solver. Let's start with a simple 4 (\*\*\*\*) star sudoku

In [1]:
# https://www.adrian.idv.hk/2019-01-30-simanneal/
# Parool Dinsdag 19 sept ****
sudoku_grid = [
    [0, 0, 0, 0, 0, 0, 0, 9, 2],
    [0, 0, 0, 1, 0, 0, 0, 4, 0],
    [9, 0, 0, 2, 4, 0, 0, 0, 7],
    [8, 0, 0, 7, 0, 0, 1, 5, 0],
    [6, 5, 0, 9, 0, 1, 0, 7, 8],
    [0, 7, 4, 0, 0, 8, 0, 0, 6],
    [3, 0, 0, 0, 9, 5, 0, 0, 1],
    [0, 8, 0, 0, 0, 6, 0, 0, 0],
    [7, 9, 0, 0, 0, 0, 0, 0, 0]
]

solution_grid = [
    [4, 1, 8, 6, 5, 7, 3, 9, 2],
    [2, 3, 7, 1, 8, 9, 6, 4, 5],
    [9, 6, 5, 2, 4, 3, 8, 1, 7],
    [8, 2, 9, 7, 6, 4, 1, 5, 3],
    [6, 5, 3, 9, 2, 1, 4, 7, 8],
    [1, 7, 4, 5, 3, 8, 9, 2, 6],
    [3, 4, 2, 8, 9, 5, 7, 6, 1],
    [5, 8, 1, 4, 7, 6, 2, 3, 9],
    [7, 9, 6, 3, 1, 2, 5, 8, 4]
]

problem = [x for row in sudoku_grid for x in row]

In [2]:
column_score = lambda n: -len(set(problem[i*9+n] for i in range(9)))
row_score = lambda n: -len(set(problem[n*9+j] for j in range(9)))
block_score = lambda n: -len(set(problem[i*9+j] for (i, j) in all_blocks[n]))

from sudoku import all_blocks


sum(column_score(n)+row_score(n) for n in range(9))

block_score(2)

-5

In [3]:
from sudoku.solvers import heuristic

heuristic.all_blocks

[[(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)],
 [(0, 3), (0, 4), (0, 5), (1, 3), (1, 4), (1, 5), (2, 3), (2, 4), (2, 5)],
 [(0, 6), (0, 7), (0, 8), (1, 6), (1, 7), (1, 8), (2, 6), (2, 7), (2, 8)],
 [(3, 0), (3, 1), (3, 2), (4, 0), (4, 1), (4, 2), (5, 0), (5, 1), (5, 2)],
 [(3, 3), (3, 4), (3, 5), (4, 3), (4, 4), (4, 5), (5, 3), (5, 4), (5, 5)],
 [(3, 6), (3, 7), (3, 8), (4, 6), (4, 7), (4, 8), (5, 6), (5, 7), (5, 8)],
 [(6, 0), (6, 1), (6, 2), (7, 0), (7, 1), (7, 2), (8, 0), (8, 1), (8, 2)],
 [(6, 3), (6, 4), (6, 5), (7, 3), (7, 4), (7, 5), (8, 3), (8, 4), (8, 5)],
 [(6, 6), (6, 7), (6, 8), (7, 6), (7, 7), (7, 8), (8, 6), (8, 7), (8, 8)]]

In [4]:
def coord(row, col):
    return row*9+col

def block_indices(block_num):
    """return linear array indices corresp to the sq block, row major, 0-indexed.
    block:
       0 1 2     (0,0) (0,3) (0,6)
       3 4 5 --> (3,0) (3,3) (3,6)
       6 7 8     (6,0) (6,3) (6,6)
    """
    firstrow = (block_num // 3) * 3
    firstcol = (block_num % 3) * 3
#    indices = [coord(firstrow+i, firstcol+j) for i in range(3) for j in range(3)]
    indices = [(firstrow+i, firstcol+j) for i in range(3) for j in range(3)]

    return indices


block_indices(4)

[(3, 3), (3, 4), (3, 5), (4, 3), (4, 4), (4, 5), (5, 3), (5, 4), (5, 5)]

In [5]:
import random

def initial_solution(problem):
    """provide sudoku problem, generate an init solution by randomly filling
    each sq block without considering row/col consistency"""
    solution = problem.copy()
    for block in range(9):
        indices = [i*9+j for (i, j) in heuristic.all_blocks[block]]
#        block = problem[indices]
        block = [problem[i] for i in indices]
        zeros = [i for i in indices if problem[i] == 0]
        to_fill = [i for i in range(1, 10) if i not in block]
        random.shuffle(to_fill)
        for index, value in zip(zeros, to_fill):
            solution[index] = value
    return solution


initial_solution(problem)

[2,
 8,
 1,
 5,
 6,
 9,
 8,
 9,
 2,
 3,
 7,
 6,
 1,
 7,
 8,
 6,
 4,
 5,
 9,
 4,
 5,
 2,
 4,
 3,
 3,
 1,
 7,
 8,
 3,
 2,
 7,
 6,
 2,
 1,
 5,
 9,
 6,
 5,
 1,
 9,
 4,
 1,
 3,
 7,
 8,
 9,
 7,
 4,
 5,
 3,
 8,
 4,
 2,
 6,
 3,
 1,
 6,
 8,
 9,
 5,
 2,
 5,
 1,
 4,
 8,
 5,
 1,
 3,
 6,
 8,
 9,
 4,
 7,
 9,
 2,
 2,
 7,
 4,
 3,
 6,
 7]

In [6]:
from simanneal import Annealer

class Sudoku_Sq(Annealer):
    def __init__(self, problem):
        self.problem = problem
        state = initial_solution(problem)
        super().__init__(state)
    def move(self):
        """randomly swap two cells in a random square"""
        block = random.randrange(9)
        indices = [(i*9+j) for (i, j) in all_blocks[block] if self.problem[i*9+j] == 0]
        m, n = random.sample(indices, 2)
        self.state[m], self.state[n] = self.state[n], self.state[m]
    def energy(self):
        """calculate the number of violations: assume all rows are OK"""
        column_score = lambda n: -len(set(self.state[coord(i, n)] for i in range(9)))
        row_score = lambda n: -len(set(self.state[coord(n, i)] for i in range(9)))
        block_score = lambda n: -len(set(self.state[i*9+j] for (i, j) in all_blocks[n]))
        score = sum(column_score(n)+row_score(n)+block_score(n) for n in range(9))
        if score == -3*9*9:
            self.user_exit = True # early quit, we found a solution
        return score


In [28]:
sudoku = Sudoku_Sq(problem)
sudoku.copy_strategy = "method"

sudoku.Tmax = 0.5
sudoku.Tmin = 0.05
sudoku.steps = 150000
sudoku.updates = 100
state, e = sudoku.anneal()
print("\n")

print("E=%f (expect -243)" % e)


 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.46663       -241.00     8.07%     1.60%     0:00:00     0:00:04



E=-243.000000 (expect -243)


In [19]:
from sudoku.printer import display_list, display_grid

display_list(state)
print()
display_grid(solution_grid)

4 1 5 | 6 3 7 | 8 9 2 
2 6 7 | 1 8 9 | 5 4 3 
9 3 8 | 2 4 5 | 6 1 7 
------+-------+------
8 2 9 | 7 6 3 | 1 5 4 
6 5 3 | 9 4 1 | 2 7 8 
1 7 4 | 5 2 8 | 9 3 6 
------+-------+------
3 4 2 | 8 9 5 | 7 6 1 
5 8 1 | 4 7 6 | 3 2 9 
7 9 6 | 3 1 2 | 4 8 5 

4 1 8 | 6 5 7 | 3 9 2 
2 3 7 | 1 8 9 | 6 4 5 
9 6 5 | 2 4 3 | 8 1 7 
------+-------+------
8 2 9 | 7 6 4 | 1 5 3 
6 5 3 | 9 2 1 | 4 7 8 
1 7 4 | 5 3 8 | 9 2 6 
------+-------+------
3 4 2 | 8 9 5 | 7 6 1 
5 8 1 | 4 7 6 | 2 3 9 
7 9 6 | 3 1 2 | 5 8 4 
